## Course 2. Symbolic Execution.
### **Exercise 2.**  Developing Symbolic Execution Tools for EVM Using Z3

The goal of this exercise is to familiarize yourself with the details of symbolic execution techniques. In contrast to [Exercise 1](https://colab.research.google.com/drive/17oyX0mL9TtpjPUQEYO4h_GwyW1A8V8r6?usp=sharing) where we have utilized the API of an existing symbolic execution tool, this exercise is focused on building our own EVM-level symbolic execution engine (and executable formal semantics for our EVM-like language) using Z3 as a constraint solver.
 <!-- (more on that later)  -->
This Jupyter notebook uses Python 3.7, and we use Python bindings of Z3 throughout the exercise.

*The exercise and the accompanying tutorial are heavily based on (1) [Z3 Tutorial](https://colab.research.google.com/github/philzook58/z3_tutorial/blob/master/Z3%20Tutorial.ipynb) by Philip Zucker and the (2) [EVM Symbolic Execution tutorial](https://github.com/williamberman/evm-symbolic-execution/blob/master/EVM%20Symbolic%20Execution.ipynb) by William Berman.*

#### **Installation**

Z3 can be installed using `pip`.

In [ ]:
!pip install z3-solver
from z3 import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 89 kB/s 


## Z3 Basics & Exercise Overview

According to its [website](https://github.com/Z3Prover/z3), Z3 is a theorem prover from Microsoft Research, which can be used to check the satisfiability of logical formulas.
<!-- over one or more theories -->
In the context of symbolic execution, it means that Z3 can be used to check path reachability (as satisfiability of constraints collected along the exploration of this path) or some properties over \[smart contract\] execution.

Z3 has official bindings for C, C++, Python, .NET, OCaml, Java. Bindings in other programming languages may also be available: for example, there are at least two Z3 bindings for Rust: high-level [z3](https://docs.rs/z3/latest/z3/) and low-level [z3-sys](https://crates.io/crates/z3-sys). Z3 also has a command line interface that accepts a logical formula input format named [SMT-LIB](https://smtlib.github.io/jSMTLIB/SMTLIBTutorial.pdf).

In this tutorial and exercise, we will use Z3's Python API, which we installed using `pip install z3-solver` in the previous cell.

Inspired by William Berman's [tutorial](https://github.com/williamberman/evm-symbolic-execution/blob/master/EVM%20Symbolic%20Execution.ipynb), in this exercise, we will gradually develop a simple symbolic execution tool for a small EVM-like language. We will test it on the `Exchange` smart contract we are using as a running example in this module. As we know, a symbolic execution engine needs to maintain a set of *path constraints* and a *symbolic state*. Let's see how we can model both of them using Z3 and its internal representation. The source code of the running example is shown below for reference:

```javascript
contract Exchange {   
    function deposit(uint _in, uint _out, uint _approved) public {
        uint balance_in = 100; uint balance_out = 100;

        if (_in >= _out && _out <= balance_out) {
            if (_approved >= _in) {
                balance_in += _in;
                _approved -= _in;
            }
            
            balance_out -= _out;
        }

        assert(balance_in + balance_out >= 200);
   }
}
```
 

#### Z3 Sorts

Z3 offers several types, called *sorts*, to declare variables—including those with symbolic values. The variables can also be used in the constraints.

In our running example, we only use `uint` variables which, in EVM, are unsigned integers of 256 bits in size. Z3 has a sort `Int` we will use in the first half of the exercise to model these variables. However, integers in Z3 are *signed* and *unbounded*. We will later see how EVM semantics of `uint` can be modeled more accurately using the bitvector (`BitVec`) sort of Z3.

Variables of `Int` sort can be declared in Z3 Python API as follows:
`x = Int('x')`. This command creates a Z3 integer variable called `x`.
To declare an `Int` numeral constant with a certain value, use the command `IntVal` and specify the value in brackets:
`y = IntVal('10')`.

**Task 1:** declare three integer variables in Z3 called `s_in`, `s_out`, and `s_appr`. These variables correspond to input parameters of the `deposit` function in our running example.
Declare two constants `balance_in` and `balance_out` representing initial values (`100`) of same-named variables in `deposit`.
You can also check the sort of the variable `s_in` by calling `s_in.sort()`.

In [ ]:
s_in = # (1) create a Z3 integer called, e.g., 'in';
# repeat that for the next two variables
s_out = 
s_appr =

balance_in = # (2) create a Z3 integer constant holding the initial value
# of 'balance_in' (100); repeat that for 'balance_out'
balance_out = 

# (3) print the sort of 'balance_out'
print(...)

In [ ]:
#@title Task 1 Solution

s_in = Int('in')
s_out = Int('out')
s_appr = Int('appr')

balance_in = IntVal('100')
balance_out = IntVal('100')

print(balance_out.sort())

### Z3 Expressions

Now that we have variables and constant values, we can use them to build expressions and constraints. In Python API, many Z3 operators (`=`, `>`, `>=`, `<`, `<=`, `==`, `!=`, `+`, `-`, etc.) are intuitive since they overload Python operators.

Ultimately, we want our symbolic execution tool (and Z3) to identify a vulnerable path in the execution of our smart contract. From [manual analysis](https://github.com/WilfredTA/formal-methods-curriculum/blob/master/courses/2_Approaches_Modeling_Verification/content/1_Symbolic_Execution/symbolic_execution.md) and [Exercise 1](https://colab.research.google.com/drive/17oyX0mL9TtpjPUQEYO4h_GwyW1A8V8r6), we already now which path leads to the violation: it takes a *true* branch at line 4 (`if (_in >= _out && _out <= balance_out)`) and a *false* branch at line 5 (`if (_approved >= _in)`, i.e., `_approved < _in` holds).

In our example, the path is considered vulnerable if the property written as as assertion (`assert(balance_in + balance_out >= 200)`) can be violated. In other words, we need Z3 to check whether the opposite (`balance_in + balance_out < 200`) is satisfiable—if it is, there is a property violation.

**Task 2**: Using the variables declared in Task 1, let's manually encode this vulnerable path in Z3 as a set of constraints (corresponding to the execution of this path) AND a vulnerable condition (`balance_in + balance_out < 200`). Then, let's use Z3 to find concrete values of `s_in`, `s_out`, `s_appr` that lead to the assertion violation.

Note that our Z3 constant `balance_out` stores the initial value of this variable (`100`). However, in a vulnerable path, the value of `balance_out` is updated in line 11 (`balance_out -= _out`). 
Therefore, `balance_out` should be updated (`balance_out -= s_out`) prior to adding the constraint corresponding to the assertion.

To complete Task 2, create an instance of a Z3 solver as `s = Solver()` and add each constraint using the `s.add(...)` command. You can add constraints (`_in >= _out`, `_out <= balance_out`, `_approved < _in`, `balance_in + balance_out - s_out < 200`) one by one.

Run `s.check()` to determine constraints satisfiability and `s.model()` to get concrete assignments for symbolic variables. Note that the *model* (i.e., concrete assignments) can only be obtained for a set of constraints that is satisfiable.

In [ ]:
s = Solver()
s.add(...) # (1) Substitute '...' with a constraint satisfied by a vulnerable path 
# (2) Add other path constraints
# (3) Update 'balance_out'
# (4) Add the negation of the property

# Check satisfiability of the resulting set of constraints
print(s.check())
# Get the model
print(s.model())

In [ ]:
#@title Task 2 Solution

s = Solver()
s.add(s_in >= s_out)
s.add(s_out <= balance_out)
s.add(s_appr < s_in)

balance_out -= s_out

# That's the negation of the property
s.add(balance_in + balance_out - s_out < 200)

print(s.check())
print(s.model())

As you can see, the "vulnerable" concrete assignments found by this Z3 script are similar to those found by Manticore in Exercise 1.
<!-- And it also uses Z3 by default -->

Z3 also supports Boolean operators, such as `And`, `Or`, `Not`, etc., which also appear in the source code we aim to analyze.

**Task 3**: use Boolean operators to build constraints that more closely resemble the implementation, i.e., `_in >= _out` **AND** `_out <= balance_out`, `NOT (_approved >= _in)`, `NOT (balance_in + balance_out >= 200)`. Use `And(expr1, expr2)` and `Not(expr)` commands for this.

In [ ]:
s = Solver()
s.add(...) # (1) Substitute '...' with a constraint satisfied by a vulnerable path;
# Add another constraints 

balance_out -= s_out

s.add(...) # (2) Add the negation of the property

print(s.check())
print(s.model())

In [ ]:
#@title Task 3 Solution

s = Solver()
s.add(And(s_in >= s_out, s_out <= balance_out))
s.add(Not(s_in >= s_appr))

balance_out -= s_out

s.add(Not(balance_in + balance_out >= 200))

print(s.check())
print(s.model())


In addititon to integers we have considered, Z3 has more sorts and operators, such as `Bool`, `Real`, `BitVec` (characterized by size such as 8, 16, ..., 256 bits), etc.
Examples of some variables of these sorts are shown below.

Arrays in Z3 are represented using the `Array` sort, which is based on a *basic theory of arrays*. In Z3, the command `Select(A, i)` returns the value stored at index `i` of the array `a`. `Store(A, i, v)` returns a new array identical to `A`, but at index `i` it contains the value `v`. In Python API, `Select(A, i)` corresponds to `A[i]`.

In Z3, you can also simplify an expression using a `simplify(...)` command.

<!-- Note, however, that arrays in Z3 are used to model unbounded or very large arrays. Arrays should not be used to model small finite collections of values. It is usually much more efficient to create different variables using list comprehensions.
More efficient solution than using an array:
X = IntVector('x', 3)
print X[0] + X[1] + X[2] >= 0
print Sum(X) >= 0
 -->

In [ ]:
x = Bool("x")
y = BoolVal(True)
print(y)

r = Real("r")
v = BitVec("v", 32) # 32-bit bitvector

a = Array("A", IntSort(), BoolSort()) # array A of Booleans indexed by Int
x = Int('x')

print(a[x])
print(Select(a, x))
# Storing 'True' at position 'x' in array 'a'
print(Store(a, x, True))
# Getting the value at position 'x' in array 'a'
print(Select(Store(a, x, True), x))
print(simplify(Select(Store(a, x, True), x)))

True
A[x]
A[x]
Store(A, x, True)
Store(A, x, True)[x]
True


### Additional Exercises

More exercises on using Z3 to prove theories and solve mathematical problems are available in Philip Zucker's [Z3 Tutorial](https://colab.research.google.com/github/philzook58/z3_tutorial/blob/master/Z3%20Tutorial.ipynb).

### EVM-like Symbolic Stack Machine Implementation

In the second part of the exercise we'll be implementing a symbolic *stack machine* similar to EVM (Ethereum Virtual Machine). While more details on stack and EVM can be found online, *stack* is a data structure that serves as a collection of elements, with two main operations: *push*, which adds an element to the \[top of the\] collection, and *pop*, which removes the most recently added element that was not yet removed. The order in which elements come off a stack gives rise to its alternative name, LIFO *(last in, first out)*. \[Source: [Wikipedia](https://en.wikipedia.org/wiki/Stack_(abstract_data_type)\].

The EVM executes as a stack machine. During execution, the EVM maintains a transient memory, which does not persist between transactions—we will use it to store local variables from our running example. The bytecode of (compiled) smart contracts executes as a number of EVM opcodes, which perform standard operations like `AND` (`&&`), `ADD` (`+`), `MUL` (`*`), etc.
\[Source: [Ethereum.org](https://ethereum.org/en/developers/docs/evm/)\].
To symbolically execute EVM bytecode of a smart contract, we need to develop operational semantics for our language, in other words—establish the relation between our language elements and its representation in Z3.
Our symbolic VM should be able to process a sequence of opcodes corresponding to a smart contract similar to how EVM would do that; however, it should also accept symbolic values as input and perform satisfiability checking for the execution paths of a smart contract.

In this exercise, we intentionally keep the number of opcodes to a minimum, however, we encourage you to experiment with a larger subset of EVM (or even perform symbolic execution directly on source code). The list of opcodes and an interactive playground can be found on the [evm.codes](https://www.evm.codes/) website.

The following code implements a small machine that can perform simple arithmetic operations, such as *adding* (`ADD`) and *subtraction* (`SUB`) between two integers that have to be placed in the stack (`PUSH`) before the operator.
In our stack machine, we implement a *program counter* (`pc`) that points to the address of the next instruction to be executed.
This code uses `dataclasses` Python module for the implementation of opcode classes. The `run` function returns the element at the top of the stack after processing the `code` (a sequence of opcodes) that is given.

In [ ]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int

class StackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        if instruction == ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
        elif instruction == SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
        elif instruction == PUSH(n=instruction.n):
                self.stack.append(instruction.n)
        else:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

We initialize this stack machine with `code`—an array of opcodes to be executed. For example, we can use it to calculate the result of following expression: `(3 + 2) - 1`. 
It is executed in the following order: first, `[expr1 =] (3 + 2)`, then, `expr1 - 1`. The last operation performed in this expression is subtraction, therefore, our opcode sequence `code` will end with a `SUB()`. `SUB()` takes two operands from the top of the stack. The result of `PUSH(a), PUSH(b), SUB()` is `b - a`.

Therefore, for `(3 + 2) - 1`, `code` should look as follows: `PUSH(1), [3 + 2], SUB()`. Similar to `SUB()`, `ADD()` performs addition between two top-stack elements. Therefore, the whole sequence would be: 
`m = StackArith([PUSH(1), PUSH(2), PUSH(3), ADD(), SUB()])`.
You can check the result of this expression below by running `m.run()`.

In [ ]:
# (3 + 2) - 1 -> 4
m = StackArith([PUSH(1), PUSH(2), PUSH(3), ADD(), SUB()])
m.run()

4

**Task 4:** Run the machine on an expression `(c * b) + a` with symbolic integer variables, e.g., `Int('a')`, `Int('b')`, and `Int('c')`, being pushed to the stack.

You can run it on both symbolic *and* concrete values.

In [ ]:
# (c - b) + a

m = StackArith([...]) # (1) Add the opcode sequence calculating the expression
m.run()

In [ ]:
#@title Task 4 Solution
# (c - b) + a

m = StackArith([PUSH(Int('a')), PUSH(Int('b')), PUSH(Int('c')), SUB(), ADD()])
m.run()

m = StackArith([PUSH(5), PUSH(IntVal('3')), PUSH(Int('c')), SUB(), ADD()])
m.run()


Let's extend the functionality of our machine so that it can process at least one *vulnerable*  execution path of our running example. To check the assertion in `deposit`, we will add an `ASSERT` opcode to our symbolic VM.

We'll also add a new field `constraints` to our machine that will capture the execution context, i.e., constraints for each executed path. We'll initialize this field with a new instance of a (Z3) `Solver`: `self.constraints = Solver()`. `ASSERT()` adds a (Boolean) expression at the top of the stack to `constraints`.

We also add path constraints to the Z3 solver along the (symbolic) execution process. Then we can use the solver to check satisfiability of the collected set of constraints upon encountering branch conditions, assertions, or other conditional statements. For now, we will do that in the end of the execution in the `run()` function, after processing of the `ASSERT()` opcode. If the constraint set is satisfiable, we ask the solver to return a *model*, i.e., satisfying assignments to symbolic variables.

Let's also add Boolean operators to build assertions. In the code below, we added the `AND` operator, which works similarly to other binary operators we discussed above.

**Task 5**: Using the examples of `AND` class definition and processing in `step()`, add binary operators less-than-or-equal `LTE`, greater-than-or-equal `GTE`, and a unary operator `NOT`.

Use a resulting machine to find solution for the following expression: `x <= (5 + 10)`.
To do so, your `code` should contain the opcodes that perform the following: (1) add `5` to `10`, (2) push `x` to the stack, (3) build a Boolean expression `x <= (5 + 10)` by using `LTE`, (4) check satisfiability of this constraint using **assert**.

In [ ]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int

@dataclass
class ASSERT:
    pass

@dataclass
class AND:
    pass

# (1) Add classes for NOT, LTE, GTE opcodes
@dataclass
class ...

class Assert_StackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        # Adding the set of constraints to the machine state
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        if instruction == ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
        elif instruction == SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
        elif instruction == AND():
                self.stack.append(And(self.stack.pop(), self.stack.pop()))
        # (2) Add handling of NOT, LTE, GTE opcodes
        # Note that NOT corresponds to (Not (expr1, expr2)) operator in Z3
        elif instruction == ASSERT():
                self.constraints.add(self.stack[-1])
        elif instruction == PUSH(n=instruction.n):
                self.stack.append(instruction.n)
        else:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        # Checking if the resulting set of constraints is satisfiable
        # Returning a model if they are
        if self.constraints.check() == sat:
            print('Contraints are satisfiable')
            model = self.constraints.model()
            return model
        else:
            print('Contraints are not satisfiable')
            # The set of constraints is not satisfiable
            return None

In [ ]:
# x <= (5 + 10)

m = Assert_StackArith([...]) # (3) Find solution for x
m.run()

In [ ]:
#@title Task 5 Solution
# x <= (5 + 10)

from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int

@dataclass
class ASSERT:
    pass

@dataclass
class AND:
    pass

@dataclass
class LTE:
  pass

@dataclass
class GTE:
  pass

@dataclass
class NOT:
  pass

class Assert_StackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        # Adding the set of constraints to the machine state
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        if instruction == ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
        elif instruction == SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
        elif instruction == AND():
                self.stack.append(And(self.stack.pop(), self.stack.pop()))
        elif instruction == LTE():
                self.stack.append(self.stack.pop() <= self.stack.pop())
        elif instruction == GTE():
                self.stack.append(self.stack.pop() >= self.stack.pop())
        elif instruction == NOT():
                self.stack.append(Not(self.stack.pop()))
        elif instruction == ASSERT():
                self.constraints.add(self.stack[-1])
        elif instruction == PUSH(n=instruction.n):
                self.stack.append(instruction.n)
        else:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        # Checking if the resulting set of constraints is satisfiable
        # Returning a model if they are
        if self.constraints.check() == sat:
            print('Contraints are satisfiable')
            model = self.constraints.model()
            return model
        else:
            # The set of constraints is not satisfiable
            print('Contraints are not satisfiable')
            return None


# x <= (5 + 10)
m = Assert_StackArith([PUSH(5), PUSH(10), ADD(), PUSH(Int('x')),
                       LTE(), ASSERT()])
m.run()

Now, if you assert an unsatisfiable expression, our symbolic machine will will not be able to find a satisfying assignment. Consider the following example:

In [ ]:
# x + 10 >= 9 && x >= 0

x = Int('x')

m = Assert_StackArith([PUSH(x), PUSH(IntVal('10')), ADD(), PUSH(IntVal('9')),
                       GTE(), PUSH(0), PUSH(x), GTE(), AND(), ASSERT()])
m.run()

Contraints are not satisfiable


We can now process the vulnerable execution path of our running example. Similar to how we did it in the first part of this exercise, we can, again, manually encode the constraints (`_in >= _out && _out <= balance_out`, `NOT (_approved >= _in)`, `NOT (balance_in + balance_out >= 200`).

**Task 6**: Run the symbolic machine we've developed so far on the vulnerable execution path of the running example `Exchange`. Don't forget to use an updated value of `balance_out`! Use `ASSERT()` to add a Boolean expression to the set of constraints. 

In [ ]:
s_in = Int('in')
s_out = Int('out')
s_appr = Int('appr')

bal_in = IntVal('100')
bal_out = IntVal('100')

                # (4) if (_in >= _out && _out <= balance_out) {
m = Assert_StackArith([..., ASSERT(),
                # (5, 9) NOT if (_approved >= _in) { ... } // { else revert(); }
                ..., ASSERT(),
                # (11) balance_out -= _out;
                ...,
                # (14) balance_in + balance_out;
                # Hint: the updated value of balance_out was calculated in the previous step
                ...,
                # (14) ASSERT NOT (balance_in + balance_out >= 200);
                ..., NOT(), ASSERT()])

m.run()

In [ ]:
#@title Task 6 Solution

s_in = Int('in')
s_out = Int('out')
s_appr = Int('appr')

bal_in = IntVal('100')
bal_out = IntVal('100')

                # (4) if (_in >= _out && _out <= balance_out) {
m = Assert_StackArith([PUSH(s_out), PUSH(s_in), GTE(), PUSH(bal_out), PUSH(s_out), LTE(), AND(), ASSERT(),
                # (5, 9) NOT if (_approved >= _in) { ... } // { else revert(); }
                PUSH(s_in), PUSH(s_appr), GTE(), NOT(), ASSERT(),
                # (11) balance_out -= _out;
                PUSH(s_out), PUSH(bal_out), SUB(),
                # (14) balance_in + balance_out
                PUSH(bal_in), ADD(),
                # (14) ASSERT NOT (balance_in + balance_out >= 200);
                PUSH(200), LTE(), NOT(), ASSERT()])

m.run()

Contraints are satisfiable


[in = 65, out = 65, appr = 64]

As we can see from the result, Z3 helps us find similar concrete values triggering a property violation again.

#### Adding Read-Only Memory to Stack Machine

Let's make the machine more realistic by adding memory to it—for now, read-only memory that can store some values during the execution of a smart contract (in fact, during the execution of one transaction). We'll pass the values stored in memory as a Python list at the initialization of the machine. These values are stored in its `memory` field: `self.memory = memory`. The memory can be read using an `MLOAD` instruction, which reads from a memory slot indexed by the top element on a stack.

In [ ]:
@dataclass
class MLOAD:
    pass

class ReadOnlyMem_Stack:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.constraints = Solver()
        # Added machine's memory
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        if instruction == ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
        elif instruction == SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
        elif instruction == LTE():
                self.stack.append(self.stack.pop() <= self.stack.pop())
        elif instruction == GTE():
                self.stack.append(self.stack.pop() >= self.stack.pop())
        elif instruction == AND():
                self.stack.append(And(self.stack.pop(),self.stack.pop()))
        elif instruction == NOT():
                self.stack.append(Not(self.stack.pop()))
        elif instruction == ASSERT():
                self.constraints.add(self.stack[-1])
        elif instruction == MLOAD():
              # MLOAD opcode reads from the memory slot
              # indexed by a value at the top of the stack
              self.stack.append(self.memory[self.stack.pop()])
        elif instruction == PUSH(n=instruction.n):
                self.stack.append(instruction.n)
        else:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        return self.stack[-1]

Let's see how that works:

In [ ]:
mem = [0 for x in range(5)]
mem[4] = 3
print(mem)
# (mem[4] + 2) - 1
# -> (3 + 2) - 1 
# -> 4
m = ReadOnlyMem_Stack([PUSH(1), PUSH(2), PUSH(4), MLOAD(), ADD(), SUB()], mem)
m.run()

[0, 0, 0, 0, 3]


4

Converting stack-based arithmetic operations from fully concrete to fully or partially symbolic was rather straightforward. Symbolic memory is more complicated because the memory can store symbolic values, the memory can be indexed by symbolic values, and even the length of the memory can be symbolic.

In general, there are different approaches to model memory in symbolic execution tools, and they all come with their own set of trade-offs (more details can be found in Section 3 of [A Survey of Symbolic Execution Techniques](https://arxiv.org/pdf/1610.00502.pdf) by Baldoni et al.).

For simplicity, we will assume that memory addresses are concrete (and we add an assertion that the memory index is concrete in `MLOAD`) but it can store symbolic values.

In [ ]:
class SymMem_Stack:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.constraints = Solver()
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        if instruction == ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
        elif instruction == SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
        elif instruction == LTE():
                self.stack.append(self.stack.pop() <= self.stack.pop())
        elif instruction == GTE():
                self.stack.append(self.stack.pop() >= self.stack.pop())
        elif instruction == AND():
                self.stack.append(And(self.stack.pop(), self.stack.pop()))
        elif instruction == NOT():
                self.stack.append(Not(self.stack.pop()))
        elif instruction == ASSERT():
                self.constraints.add(self.stack[-1])
        elif instruction == MLOAD():
              # Getting the requested index in memory
              idx = self.stack.pop()
              assert isinstance(idx, int), 'MLOAD: requires concrete index'
              # Pushing the value read from this slot to stack
              self.stack.append(self.memory[idx])
        elif instruction == PUSH(n=instruction.n):
                self.stack.append(instruction.n)
        else:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        print(self.constraints)
        if self.constraints.check() == sat:
            print('Constraints are satisfiable')
            model = self.constraints.model()
            return model
        else:
            print('Constraints are not satisfiable')
            return None

In [ ]:
# z + 2 - 1 >= 4 <=> mem[4] + 2 - 1 >= 4

mem = [0 for x in range(5)]
mem[4] = Int('z')
print(mem)

m = SymMem_Stack([PUSH(1), PUSH(2), PUSH(4), MLOAD(), ADD(), SUB(), PUSH(4), LTE(), ASSERT()], mem)

m.run()

[0, 0, 0, 0, z]
[z + 2 - 1 >= 4]
Constraints are satisfiable


[z = 3]

If the memory address is symbolic, the machine will report an error.

In [ ]:
mem = [0 for x in range(5)]
m = SymMem_Stack([PUSH(Int('x')), MLOAD()], mem)
try:
    m.run()
except AssertionError as e:
    print(e)

MLOAD: requires concrete index


#### Making Memory Writable Using Arrays

To make memory writeable, we will use `Array` sort of Z3 we discussed in the first part of the exercise. In a machine, `memory` will now be an integer array called "memory" indexed by integers (`self.memory = Array('memory', IntSort(), IntSort())`).

We'll add an additional `MSTORE` instruction that reads (1) an `index` and (2) a `value` from stack, and writes `value` to the `index` slot in the memory array using a `Store` operation (`self.memory = Store(self.memory, idx, val)`).

Notice that this writeable memory here corresponds to the *symbolic memory state* maintained by a symbolic execution engine. This state is updated through the execution of assignments.

In [ ]:
@dataclass
class MSTORE:
    pass

class SymMem_Stack:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        self.constraints = Solver()
        # Memory is now a Z3 array
        self.memory = Array('memory', IntSort(), IntSort())
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        if instruction == ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
        elif instruction == SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
        elif instruction == LTE():
                self.stack.append(self.stack.pop() <= self.stack.pop())
        elif instruction == GTE():
                self.stack.append(self.stack.pop() >= self.stack.pop())
        elif instruction == AND():
                self.stack.append(And(self.stack.pop(), self.stack.pop()))
        elif instruction == NOT():
                self.stack.append(Not(self.stack.pop()))
        elif instruction == ASSERT():
                self.constraints.add(self.stack[-1])
        elif instruction == MLOAD():
              idx = self.stack.pop()
              assert isinstance(idx, int), 'MLOAD: requires concrete index'
              self.stack.append(self.memory[idx])
        elif instruction == MSTORE():
                idx = self.stack.pop()
                val = self.stack.pop()
                # Writing val to memory[idx]
                self.memory = Store(self.memory, idx, val)
        elif instruction == PUSH(n=instruction.n):
                self.stack.append(instruction.n)
        else:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        if self.constraints.check() == sat:
            print('Constraints are satisfiable')
            model = self.constraints.model()
            return model
        else:
            print('Constraints are not satisfiable')
            return None

In [ ]:
# (1) memory[0] = x
# (2) 4 <= (memory[0] + 2) - 1

m = SymMem_Stack([
    PUSH(1),
    PUSH(2),
    # memory[0] = x
    PUSH(Int('x')), PUSH(0), MSTORE(),
    # memory[0]
    PUSH(0), MLOAD(),
    ADD(), 
    SUB(), 
    PUSH(4), LTE(), ASSERT()])

m.run()

Constraints are satisfiable


[x = 3]

**Task 7**: Similarly, find the solution for the following expression:
```
# memory[1] = y
# 5 >= (memory[1] - 10) + 1
```

In [ ]:
# (1) memory[1] = y
# (2) 5 >= (memory[1] - 10) + 1

m = SymMem_Stack([
    ..., # Insert the opcodes for the expression
    ASSERT()]) 

m.run()

In [ ]:
#@title Task 7 Solution

# (1) memory[1] = y
# (2) 5 >= (memory[1] - 10) + 1

m = SymMem_Stack([
    PUSH(1),
    PUSH(10),
    # memory[0] = x
    PUSH(Int('y')), PUSH(1), MSTORE(),
    # memory[0]
    PUSH(1), MLOAD(),
    SUB(), 
    ADD(), 
    PUSH(5), GTE(), ASSERT()])

m.run()

#### Introducing Conditional Jumps

The key functionality of symbolic execution is its ability to explore multiple execution paths at the same time. To add this feature to our symbolic tool, we need EVM's `JUMPI` instruction. This instruction reads the Boolean value from the top of the stack and, depending on whether it is *true* or *false*, jumps to a certain pc. In fact, `JUMPI` reads two values from the stack: a condition and a destination (`pc` number). In our implementation, the jump occurs if the condition does not hold.

We'll also add a `STOP` instruction: it will just move `pc` to the end of the code sequence.

In [ ]:
@dataclass
class JUMPI:
    pass

@dataclass
class STOP:
    pass

class Cond_Stack:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        self.constraints = Solver()
        self.memory = Array('memory', IntSort(), IntSort())
        
    def step(self):
        instruction = self.code[self.pc]

        if instruction == JUMPI():
            dest = self.stack.pop()
            cond = self.stack.pop()
            # If the condition holds, move on to the next statement
            # Otherwise — jump to the destination 'dest' 
            self.pc = self.pc + 1 if cond else dest
            return
           
        self.pc += 1
        if instruction == ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
        elif instruction == SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
        elif instruction == LTE():
                self.stack.append(self.stack.pop() <= self.stack.pop())
        elif instruction == GTE():
                self.stack.append(self.stack.pop() >= self.stack.pop())
        elif instruction == AND():
                self.stack.append(And(self.stack.pop(), self.stack.pop()))
        elif instruction == NOT():
                self.stack.append(Not(self.stack.pop()))
        elif instruction == ASSERT():
                self.constraints.add(self.stack[-1])
        elif instruction == MLOAD():
              idx = self.stack.pop()
              assert isinstance(idx, int), 'MLOAD: requires concrete index'
              self.stack.append(self.memory[idx])
        elif instruction == MSTORE():
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory = Store(self.memory, idx, val)
        elif instruction == STOP():
                # STOP moves the pc to the end of the execution
                self.pc = len(self.code)
        elif instruction == PUSH(n=instruction.n):
                self.stack.append(instruction.n)
        else:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        print(self.stack[-1])

We'll illustrate how it works on concrete values. Consider the following example: if `10 <= (4 - (2 + 3) - 1)` holds, `100` is being pushed to the stack, and the execution stops. If not (as it does), `200` is pushed to the stack.

In [ ]:
# (1) (2 + 3) - 1 = 4
# (2) 4 - 4 = 0
# (3) 10 <= (4 - (2 + 3) - 1)?
m = Cond_Stack([
    PUSH(1),  # Instruction #0
    PUSH(2),  # Instruction #1
    PUSH(3),  # ... 
    ADD(), 
    SUB(),
    PUSH(4),
    SUB(),
    PUSH(10),
    # Is 10 <= 4 - (2 + 3) - 1?
    LTE(),
    # if not, set pc to 13
    # (push 200)
    PUSH(13),
    JUMPI(),
    # The condition did hold
    PUSH(100),
    STOP(),
    # The next pc is 13
    # The condition did hold
    PUSH(200)
])

m.run()

200


The value at the top of the stack is `200`, as we expected. Try changing one of the concrete values (e.g., 4 to 40) in a way that causes another branch to execute, and check the result—the value on the stack should be `100`.

In [ ]:
# (1) (2 + 3) - 1 = 4
# (2) 4 - 4 = 0
# (3) 10 <= 4 - (2 + 3) - 1?
m = Cond_Stack([
    PUSH(1),  # Instruction #0
    PUSH(2),  # Instruction #1
    PUSH(3),  # ... 
    ADD(), 
    SUB(),
    PUSH(...), # (1) Insert some value that makes '10 <= 4 - (2 + 3) - 1' hold
    SUB(),
    PUSH(10),
    # Is 10 <= 4 - (2 + 3) - 1?
    LTE(),
    # if not, set pc to 13
    # (push 200)
    PUSH(13),
    JUMPI(),
    # The condition did hold
    PUSH(100),
    STOP(),
    # The next pc is 13
    # The condition did hold
    PUSH(200) # Instruction #13
])

m.run()

Our machine can work with conditional jumps, meaning that it can analyze *branches* in the execution path. But what if the conditional argument to the jump is symbolic? Symbolic here means that it can either be true of false, both of these conditions are satisfiable under some assignments. In this case, our symbolic machine must analyze both paths, provided that their respective constraints are satisfiable. This means that our machine must now keep track of multiple execution contexts (corresponding to all branches taken). To allow that, let's define a new class `MachineContext` that will store the information about a machine's execution context: its pc, stack, memory, constraints. The `code` is shared by all machines, so we leave it in our execution engine class.

Upon every conditional jump, we will split the context into 2 (i.e., fork the execution) by making a copy of the current context. One of them would take the *true* branch and another—the *false* branch, which will be reflected in their `constraints`.
This implementation explores the execution tree produced by symbolic execution in the DFS ([depth-first search](https://en.wikipedia.org/wiki/Depth-first_search)) manner.

In [ ]:
@dataclass
class MachineContext:
    pc: int
    stack: any
    memory: any
    constraints: any
        
    def __copy__(self):
        return MachineContext(
            pc=self.pc,
            stack=self.stack.copy(),
            # Memory is represented as an SMT Array which is immutable
            # and does not need to be copied
            memory=self.memory,
            constraints=copy.copy(self.constraints)
        )

class SymCond:
    def __init__(self, code):
        self.code = code
        # Now multiple machine contexts can be stored
        self.machines = [
            MachineContext(
                pc=0, 
                stack=[], 
                memory=Array('memory', IntSort(), IntSort()), 
                constraints=Solver()
            )
        ]
        
    def step(self, machine):
        instruction = self.code[machine.pc]

        # JUMPI now creates two different machine contexts        
        if instruction == JUMPI():
            dest = machine.stack.pop()
            assert isinstance(dest, int), 'JUMPI: requires concrete destination'
            cond = machine.stack.pop()
            other_machine = copy.copy(machine)
            
            # Another machine does take the jump
            other_machine.pc = dest
            # Its constraints are updated accordingly
            other_machine.constraints.append(Not(cond))
            
            # The current machine does not take the jump
            machine.pc = machine.pc + 1
            machine.constraints.append(cond)
            
            self.machines.append(other_machine)

            return        
           
        machine.pc += 1
        if instruction == ADD():
                machine.stack.append(machine.stack.pop() + machine.stack.pop())
        elif instruction == SUB():
                machine.stack.append(machine.stack.pop() - machine.stack.pop())
        elif instruction == LTE():
                machine.stack.append(machine.stack.pop() <= machine.stack.pop())
        elif instruction == GTE():
                machine.stack.append(machine.stack.pop() >= machine.stack.pop())
        elif instruction == AND():
                machine.stack.append(And(machine.stack.pop(), machine.stack.pop()))
        elif instruction == NOT():
                machine.stack.append(Not(machine.stack.pop()))
        elif instruction == ASSERT():
                machine.constraints.add(machine.stack[-1])
        elif instruction == MLOAD():
              idx = machine.stack.pop()
              assert isinstance(idx, int), 'MLOAD: requires concrete index'
              machine.stack.append(machine.memory[idx])
        elif instruction == MSTORE():
                idx = machine.stack.pop()
                val = machine.stack.pop()
                machine.memory = Store(machine.memory, idx, val)
        elif instruction == STOP():
                machine.pc = len(self.code)
        elif instruction == PUSH(n=instruction.n):
                machine.stack.append(instruction.n)
        else:
                assert False

    def choose_machine(self):
        machine = None
        
        for m in self.machines:
            if m.pc < len(self.code):
                machine = m
                break
                
        return machine        

    def run(self):
        machine = self.choose_machine()
        
        while machine:
            self.step(machine)
            machine = self.choose_machine()            
                
        for m in self.machines:
          if m.constraints.check() == sat:
              print('Constraints are satisfiable')
              model = m.constraints.model()
              print(m.stack[-1])
              print(model)
          else:
              print('Constraints are not satisfiable')

**Task 8**: modify our previous example (`10 <= (4 - (2 + 3) - 1)`) so that it has symbolic arguments (`(w <= u - (x + y) - z)`). Run our tool on it: you should be able to see that our tool now explores both paths and generates concrete values that force the execution to take either of these paths.

In [ ]:
m = SymCond([
    PUSH(...), # (1) Change previously concrete arguments to symbolic
    PUSH(...), 
    PUSH(...), 
    ADD(), 
    SUB(),
    PUSH(...),
    SUB(),
    PUSH(...),
    LTE(),
    PUSH(13),
    JUMPI(),
    PUSH(100),
    STOP(),
    # pc = 12
    PUSH(200)
])
m.run()

In [ ]:
#@title Task 8 Solution

m = SymCond([
    PUSH((Int('x'))), 
    PUSH(Int('y')), 
    PUSH(Int('z')), 
    ADD(), 
    SUB(),
    PUSH(Int('u')),
    SUB(),
    PUSH(Int('w')),
    # if not, set pc to 13 (push 200)
    LTE(),
    PUSH(13),
    JUMPI(),
    # The condition did hold
    PUSH(100),
    STOP(),
    # pc = 12
    # The condition did not hold
    PUSH(200)
])
m.run()

At the moment, we check satisfiability of a resulting set of constraints at the end of the execution path. However, a more common approach is to perform *eager evaluation* of path constraints when their satisfiability is (eagerly) checked when each branch is taken.

**Task 9**: Add eager constraint evaluation to the processing of the `JUMPI` instruction. If the resulting path conditions are UNSAT, there is no point in analyzing this execution path further: it is unreachable and cannot be taken. Therefore, the context with these constraints can be discarded.

Since `constraints` are implemented as an instance of `z3.Solver`, their satisfiability can be checked by running `constraints.check()`. The result is `z3.sat` if the set of constraints is satisfiable, `z3.unsat` — if it isn't, and `z3.unknown` if Z3 couldn't solve a system of constraints. 

In [ ]:
@dataclass
class MachineContext:
    pc: int
    stack: any
    memory: any
    constraints: any
        
    def __copy__(self):
        return MachineContext(
            pc=self.pc,
            stack=self.stack.copy(),
            memory=self.memory,
            constraints=copy.copy(self.constraints)
        )
      

class Opt_SymCond:
    def __init__(self, code):
        self.code = code
        
        self.machines = [
            MachineContext(
                pc=0, 
                stack=[], 
                memory=Array('memory', IntSort(), IntSort()), 
                constraints=Solver()
            )
        ]
        
    def step(self, machine):
        instruction = self.code[machine.pc]

        if instruction == JUMPI():
            dest = machine.stack.pop()
            assert isinstance(dest, int), 'JUMPI: requires concrete destination'
            cond = machine.stack.pop()
            
            other_machine = copy.copy(machine)
            
            other_machine.pc = dest
            other_machine.constraints.append(Not(cond))
            
            machine.pc = machine.pc + 1
            machine.constraints.append(cond)
            
            # (1) Check if constraints for the current, 'true', branch context are UNSAT
            # (2) If they are — drop the machine context from the list of machines (self.machines)
            
            # (3) Check if constraints for the new, 'false', branch context are SAT
            # (4) If they are — add the new context to the list of machines

            return        

        machine.pc += 1
        if instruction == ADD():
                machine.stack.append(machine.stack.pop() + machine.stack.pop())
        elif instruction == SUB():
                machine.stack.append(machine.stack.pop() - machine.stack.pop())
        elif instruction == LTE():
                machine.stack.append(machine.stack.pop() <= machine.stack.pop())
        elif instruction == GTE():
                machine.stack.append(machine.stack.pop() >= machine.stack.pop())
        elif instruction == AND():
                machine.stack.append(And(machine.stack.pop(), machine.stack.pop()))
        elif instruction == NOT():
                machine.stack.append(Not(machine.stack.pop()))
        elif instruction == ASSERT():
                machine.constraints.add(machine.stack[-1])
        elif instruction == MLOAD():
              idx = machine.stack.pop()
              assert isinstance(idx, int), 'MLOAD: requires concrete index'
              machine.stack.append(machine.memory[idx])
        elif instruction == MSTORE():
                idx = machine.stack.pop()
                val = machine.stack.pop()
                machine.memory = Store(machine.memory, idx, val)
        elif instruction == STOP():
                machine.pc = len(self.code)
        elif instruction == PUSH(n=instruction.n):
                machine.stack.append(instruction.n)
        else:
                assert False

    def choose_machine(self):
        machine = None
        
        for m in self.machines:
            if m.pc < len(self.code):
                machine = m
                break
                
        return machine        

    def run(self):
        machine = self.choose_machine()
        
        while machine:
            self.step(machine)
            machine = self.choose_machine()            
                
        for m in self.machines:
          if m.constraints.check() == sat:
              print('Constraints are satisfiable')
              model = m.constraints.model()
              print(model)
          else:
              print('Constraints are not satisfiable')

In [ ]:
#@title Task 9 Solution

@dataclass
class MachineContext:
    pc: int
    stack: any
    memory: any
    constraints: any
        
    def __copy__(self):
        return MachineContext(
            pc=self.pc,
            stack=self.stack.copy(),
            memory=self.memory,
            constraints=copy.copy(self.constraints)
        )
      

class Opt_SymCond:
    def __init__(self, code):
        self.code = code
        
        self.machines = [
            MachineContext(
                pc=0, 
                stack=[], 
                memory=Array('memory', IntSort(), IntSort()), 
                constraints=Solver()
            )
        ]
        
    def step(self, machine):
        instruction = self.code[machine.pc]

        if instruction == JUMPI():
            dest = machine.stack.pop()
            assert isinstance(dest, int), 'JUMPI: requires concrete destination'
            cond = machine.stack.pop()
            
            other_machine = copy.copy(machine)
            
            other_machine.pc = dest
            other_machine.constraints.append(Not(cond))
            
            machine.pc = machine.pc + 1
            machine.constraints.append(cond)
            
            # Drop this machine context from the list if its constraints are UNSAT
            if machine.constraints.check() == unsat:
                self.machines.remove(machine)
            
            # Add other_machine only if its constraints are SAT
            if other_machine.constraints.check() == sat:
                self.machines.append(other_machine)

            return        

        machine.pc += 1
        if instruction == ADD():
                machine.stack.append(machine.stack.pop() + machine.stack.pop())
        elif instruction == SUB():
                machine.stack.append(machine.stack.pop() - machine.stack.pop())
        elif instruction == LTE():
                machine.stack.append(machine.stack.pop() <= machine.stack.pop())
        elif instruction == GTE():
                machine.stack.append(machine.stack.pop() >= machine.stack.pop())
        elif instruction == AND():
                machine.stack.append(And(machine.stack.pop(), machine.stack.pop()))
        elif instruction == NOT():
                machine.stack.append(Not(machine.stack.pop()))
        elif instruction == ASSERT():
                machine.constraints.add(machine.stack[-1])
        elif instruction == MLOAD():
              idx = machine.stack.pop()
              assert isinstance(idx, int), 'MLOAD: requires concrete index'
              machine.stack.append(machine.memory[idx])
        elif instruction == MSTORE():
                idx = machine.stack.pop()
                val = machine.stack.pop()
                machine.memory = Store(machine.memory, idx, val)
        elif instruction == STOP():
                machine.pc = len(self.code)
        elif instruction == PUSH(n=instruction.n):
                machine.stack.append(instruction.n)
        else:
                assert False

    def choose_machine(self):
        machine = None
        
        for m in self.machines:
            if m.pc < len(self.code):
                machine = m
                break
                
        return machine        

    def run(self):
        machine = self.choose_machine()
        
        while machine:
            self.step(machine)
            machine = self.choose_machine()            
                
        for m in self.machines:
          if m.constraints.check() == sat:
              print('Constraints are satisfiable')
              model = m.constraints.model()
              print(model)
          else:
              print('Constraints are not satisfiable')

Let's test the new version of our tool on the same example as before. The result of the execution should be the same as before we introduced eager evaluation.

In [ ]:
m = Opt_SymCond([
    PUSH((Int('x'))), 
    PUSH(Int('y')), 
    PUSH(Int('z')), 
    ADD(), 
    SUB(),
    PUSH(Int('u')),
    SUB(),
    PUSH(Int('w')),
    # if not, set pc to 13 (push 200)
    LTE(),
    PUSH(13),
    JUMPI(),
    # The condition did hold
    PUSH(100),
    STOP(),
    # pc = 12
    # The condition did not hold
    PUSH(200)
])
m.run()

Constraints are satisfiable
[w = 0, u = 0, z = 0, y = 0, x = 0]
Constraints are satisfiable
[w = 1, u = 0, z = 0, y = 0, x = 0]


We can use our symbolic execution tools to analyze our running example `Exchange`.

**Task 10**: Add our simplified-EVM bytecode representation of the `deposit` function we're analyzing. Run the latest `Opt_SymCond` version of our symbolic machine and analyze the result. The code of `deposit` is given below:

```cpp
 1     uint balance_in = 100; uint balance_out = 100;
 2 
 3     if (_in >= _out && _out <= balance_out) {
 4         if (_approved >= _in) {
 5             balance_in += _in;
 6             _approved -= _in;
 7         }
 8         // { else revert(); }
 9 
10         balance_out -= _out;
11     }
12     
13     assert(balance_in + balance_out >= 200);
```

Note that this code has two conditional jumps (corresponding to branches in lines 3 and 4), it updates memory by assigning values to variables (`_approved` in line 6, `balance_out` in lines 1, 10, etc.), and it ends with an `ASSERT` (and, then, optionally, `STOP`).
To store variables in memory, you can choose arbitrary slots indices, e.g., `PUSH(bal_in), PUSH(0), MSTORE()` will put the value of `bal_in` in memory slot 0.

In [ ]:
s_in = Int('in')
s_out = Int('out')
s_appr = Int('appr')

bal_in = IntVal('100')
bal_out = IntVal('100')

m = Opt_SymCond([
    # (Line 1) uint balance_in = 100;
    # uint balance_out = 100;

    # (Line 3) if (_in >= _out && _out <= balance_out) {
    # If the condition isn't satisfied, JUMP to the instruction corresponding
    # to line 13 in the above code, i.e., the assertion 
    # If it holds — continue with
    # (Line 4) if (_approved >= _in) {
    # If the condition isn't satisfied, JUMP to the instruction corresponding
    # to line 10
    # If it holds — continue with
    # (Line 5) balance_in += _in;
    # (Line 6) _approved -= _in;

    # (Line 11) balance_out -= _out;

    # (Line 14) ASSERT NOT (balance_in + balance_out >= 200);
    ])
  
m.run()

In [ ]:
#@title Task 10 Solution

s_in = Int('in')
s_out = Int('out')
s_appr = Int('appr')

bal_in = IntVal('100')
bal_out = IntVal('100')

m = Opt_SymCond([
    # (Line 1) uint balance_in = 100; uint balance_out = 100;
    PUSH(bal_in), PUSH(0), MSTORE(),    
    PUSH(bal_out), PUSH(1), MSTORE(),
    # (Line 3) if (_in >= _out && _out <= balance_out) {
    PUSH(s_out), PUSH(s_in), GTE(), PUSH(1), MLOAD(), PUSH(s_out), LTE(), AND(),
    # If the condition isn't satisfied, we jump to line 13
    PUSH(38),
    JUMPI(), # 15
    # (Line 4) if (_approved >= _in) {
    PUSH(s_in), PUSH(s_appr), GTE(),
    PUSH(32),
    JUMPI(), # 20
    # (Line 5) balance_in += _in;
    PUSH(0), MLOAD(), PUSH(s_in), ADD(), PUSH(0), MSTORE(),

    # (Line 6) _approved -= _in;
    PUSH(s_appr), PUSH(s_in), SUB(), PUSH(3), MSTORE(), # 31
  
    # (Line 11) balance_out -= _out;
    PUSH(s_out), PUSH(1), MLOAD(), SUB(), PUSH(1), MSTORE(), # MSTORE() is instruction #35

    # (Line 14) ASSERT NOT (balance_in + balance_out >= 200);
    PUSH(0), MLOAD(), PUSH(1), MLOAD(), ADD(),
    PUSH(200), LTE(), NOT(), ASSERT()
    ])
  
m.run()

#### Modeling `uints` using BitVectors

If you code works correctly, Z3 should find similar concrete values leading to the assertion violation as before. However, if you recall Exercise 1, Manticore reported other vulnerable paths where the violation was caused by **integer overflow**. We don't get those when running our tool—that is caused by using (unbounded and signed) Z3 integers that do not have the same semantics as unsigned integers in EVM and, in particular, do not overflow. 

As a final step of our exercise, we can make our tool more practical by using the `BitVec` sort of Z3 to model variables of `uint` type during symbolic execution. The function `BitVec('x', 256)` creates a bit-vector variable in Z3 called `x` with 256 bits. Similar to other sorts, the function `BitVecVal(10, 256)` creates a bitvector of size 256 containing the value 10. Note that operations on large bitvectors (such as 256) may cause scalability issues—in this example, you can experiment with various sizes (8, 16, 32, 64, 128, 256) and observe the effect they have on the execution speed. To make visual analysis of results easier, we can start by using bitvectors of size 8.

Z3 also provides *unsigned* versions of arithmetical operations we have used previously. In Z3Py, the operators `<`, `<=`, `>`, `>=`, `/`, `%` and `>>` correspond to the signed versions. The corresponding unsigned operators are `ULT`, `ULE`, `UGT`, `UGE`, `UDiv`, `URem` and `LShR`.


**Task 11**: Substitute sort `Int` with `BitVec` of size **8** (or another) in the symbolic machine implementation and `deposit` opcodes. Instead of signed operators `<=` and `>=`, use their unsigned counterparts `ULE` and `UGE`. You can add separate instructions (opcodes) for them in the machine implementation or simply change the Z3 operator that is being used when `LTE()` or `GTE()` is executed.

*Hint*: the `memory` array should now store values of bitvector sort (`BitVecSort(8)`).

In [ ]:
@dataclass
class MachineContext:
    pc: int
    stack: any
    memory: any
    constraints: any
        
    def __copy__(self):
        return MachineContext(
            pc=self.pc,
            stack=self.stack.copy(),
            memory=self.memory,
            constraints=copy.copy(self.constraints)
        )
      

class BV_SymCond:
    def __init__(self, code):
        self.code = code
        
        self.machines = [
            MachineContext(
                pc=0, 
                stack=[], 
                # (1) Update the Sort of values stored in memory
                memory=Array('memory', IntSort(), ...), 
                constraints=Solver()
            )
        ]
        
    def step(self, machine):
        instruction = self.code[machine.pc]
        if instruction == JUMPI():
            dest = machine.stack.pop()
            assert isinstance(dest, int), 'JUMPI: requires concrete destination'
            cond = machine.stack.pop()
            
            other_machine = copy.copy(machine)
            
            other_machine.pc = dest
            other_machine.constraints.append(Not(cond))
            
            machine.pc = machine.pc + 1
            machine.constraints.append(cond)
            
            if machine.constraints.check() == unsat:
                self.machines.remove(machine)
            
            if other_machine.constraints.check() == sat:
                self.machines.append(other_machine)

            return        

        machine.pc += 1
        if instruction == ADD():
                machine.stack.append(machine.stack.pop() + machine.stack.pop())
        elif instruction == SUB():
                machine.stack.append(machine.stack.pop() - machine.stack.pop())
        elif instruction == LTE():
                # (2) Use unsigned operator for <= 
                machine.stack.append(...)
        elif instruction == GTE():
                # (3) Use unsigned operator for >= 
                machine.stack.append(...)
        elif instruction == AND():
                machine.stack.append(And(machine.stack.pop(), machine.stack.pop()))
        elif instruction == NOT():
                machine.stack.append(Not(machine.stack.pop()))
        elif instruction == ASSERT():
                machine.constraints.add(machine.stack[-1])
        elif instruction == MLOAD():
              idx = machine.stack.pop()
              assert isinstance(idx, int), 'MLOAD: requires concrete index'
              machine.stack.append(machine.memory[idx])
        elif instruction == MSTORE():
                idx = machine.stack.pop()
                val = machine.stack.pop()
                machine.memory = Store(machine.memory, idx, val)
        elif instruction == STOP():
                machine.pc = len(self.code)
        elif instruction == PUSH(n=instruction.n):
                machine.stack.append(instruction.n)
        else:
                assert False

    def choose_machine(self):
        machine = None
        
        for m in self.machines:
            if m.pc < len(self.code):
                machine = m
                break
                
        return machine        

    def run(self):
        machine = self.choose_machine()
        
        while machine:
            self.step(machine)
            machine = self.choose_machine()            
        
        for m in self.machines:
          if m.constraints.check() == sat:
              print('Constraints are satisfiable, assert can be violated')
              model = m.constraints.model()
              print(model)
          else:
              print('Constraints are not satisfiable, assert can\'t be violated')            

In [ ]:
s_in = # (1) Update the Sort of symbolic (uint) variables
s_out = 
s_appr = 

bal_in =  # (2) Update the Sort of (uint constants)
bal_out = 

m = BV_SymCond([
    # (Line 1) uint balance_in = 100; uint balance_out = 100;
    PUSH(bal_in), PUSH(0), MSTORE(),    
    PUSH(bal_out), PUSH(1), MSTORE(),
    # (Line 3) if (_in >= _out && _out <= balance_out) {
    PUSH(s_out), PUSH(s_in), GTE(), PUSH(1), MLOAD(), PUSH(s_out), LTE(), AND(),
    # If the condition isn't satisfied, we jump to line 13
    PUSH(38),
    JUMPI(), # 15
    # (Line 4) if (_approved >= _in) {
    PUSH(s_in), PUSH(s_appr), GTE(),
    PUSH(32),
    JUMPI(), # 20
    # (Line 5) balance_in += _in;
    PUSH(0), MLOAD(), PUSH(s_in), ADD(), PUSH(0), MSTORE(),

    # (Line 6) _approved -= _in;
    PUSH(s_appr), PUSH(s_in), SUB(), PUSH(3), MSTORE(), # 31
  
    # (Line 11) balance_out -= _out;
    PUSH(s_out), PUSH(1), MLOAD(), SUB(), PUSH(1), MSTORE(), # MSTORE() is instruction #35

    # (Line 14) ASSERT NOT (balance_in + balance_out >= 200);
    PUSH(0), MLOAD(), PUSH(1), MLOAD(), ADD(),
    PUSH(200), LTE(), NOT(), ASSERT(),
    ])

m.run()

In [ ]:
#@title Task 11 Solution — Machine Implementation
@dataclass
class MachineContext:
    pc: int
    stack: any
    memory: any
    constraints: any
        
    def __copy__(self):
        return MachineContext(
            pc=self.pc,
            stack=self.stack.copy(),
            memory=self.memory,
            constraints=copy.copy(self.constraints)
        )
      

class BV_SymCond:
    def __init__(self, code):
        self.code = code
        
        self.machines = [
            MachineContext(
                pc=0, 
                stack=[], 
                memory=Array('memory', IntSort(), BitVecSort(8)), 
                constraints=Solver()
            )
        ]
        
    def step(self, machine):
        instruction = self.code[machine.pc]
        if instruction == JUMPI():
            dest = machine.stack.pop()
            assert isinstance(dest, int), 'JUMPI: requires concrete destination'
            cond = machine.stack.pop()
            
            other_machine = copy.copy(machine)
            
            other_machine.pc = dest
            other_machine.constraints.append(Not(cond))
            
            machine.pc = machine.pc + 1
            machine.constraints.append(cond)
            
            if machine.constraints.check() == unsat:
                self.machines.remove(machine)
            
            if other_machine.constraints.check() == sat:
                self.machines.append(other_machine)

            return        

        machine.pc += 1
        if instruction == ADD():
                machine.stack.append(machine.stack.pop() + machine.stack.pop())
        elif instruction == SUB():
                machine.stack.append(machine.stack.pop() - machine.stack.pop())
        elif instruction == LTE():
                machine.stack.append(ULE(machine.stack.pop(), machine.stack.pop()))
        elif instruction == GTE():
                machine.stack.append(UGE(machine.stack.pop(), machine.stack.pop()))
        elif instruction == AND():
                machine.stack.append(And(machine.stack.pop(), machine.stack.pop()))
        elif instruction == NOT():
                machine.stack.append(Not(machine.stack.pop()))
        elif instruction == ASSERT():
                machine.constraints.add(machine.stack[-1])
        elif instruction == MLOAD():
              idx = machine.stack.pop()
              assert isinstance(idx, int), 'MLOAD: requires concrete index'
              machine.stack.append(machine.memory[idx])
        elif instruction == MSTORE():
                idx = machine.stack.pop()
                val = machine.stack.pop()
                machine.memory = Store(machine.memory, idx, val)
        elif instruction == STOP():
                machine.pc = len(self.code)
        elif instruction == PUSH(n=instruction.n):
                machine.stack.append(instruction.n)
        else:
                assert False

    def choose_machine(self):
        machine = None
        
        for m in self.machines:
            if m.pc < len(self.code):
                machine = m
                break
                
        return machine        

    def run(self):
        machine = self.choose_machine()
        
        while machine:
            self.step(machine)
            machine = self.choose_machine()            
        
        for m in self.machines:
          if m.constraints.check() == sat:
              print('Constraints are satisfiable, assert can be violated')
              model = m.constraints.model()
              print(model)
          else:
              print('Constraints are not satisfiable, assert can\'t be violated')            

In [ ]:
#@title Task 11 Solution — `Deposit` code

s_in = BitVec('in', 8)
s_out = BitVec('out', 8)
s_appr = BitVec('appr', 8)

bal_in = BitVecVal('100', 8)
bal_out = BitVecVal('100', 8)

m = BV_SymCond([
    # (Line 1) uint balance_in = 100; uint balance_out = 100;
    PUSH(bal_in), PUSH(0), MSTORE(),    
    PUSH(bal_out), PUSH(1), MSTORE(),
    # (Line 3) if (_in >= _out && _out <= balance_out) {
    PUSH(s_out), PUSH(s_in), GTE(), PUSH(1), MLOAD(), PUSH(s_out), LTE(), AND(),
    # If the condition isn't satisfied, we jump to line 13
    PUSH(38),
    JUMPI(), # 15
    # (Line 4) if (_approved >= _in) {
    PUSH(s_in), PUSH(s_appr), GTE(),
    PUSH(32),
    JUMPI(), # 20
    # (Line 5) balance_in += _in;
    PUSH(0), MLOAD(), PUSH(s_in), ADD(), PUSH(0), MSTORE(),

    # (Line 6) _approved -= _in;
    PUSH(s_appr), PUSH(s_in), SUB(), PUSH(3), MSTORE(), # 31
  
    # (Line 11) balance_out -= _out;
    PUSH(s_out), PUSH(1), MLOAD(), SUB(), PUSH(1), MSTORE(), # MSTORE() is instruction #35

    # (Line 14) ASSERT NOT (balance_in + balance_out >= 200);
    PUSH(0), MLOAD(), PUSH(1), MLOAD(), ADD(),
    PUSH(200), LTE(), NOT(), ASSERT(),
    STOP()
    ])

m.run()

Constraints are satisfiable, assert can be violated
[in = 139, appr = 192, out = 80]
Constraints are not satisfiable, assert can't be violated
Constraints are satisfiable, assert can be violated
[in = 33, appr = 0, out = 2]


As you may notice, our tool now finds two vulnerable paths, with the first being caused by an overflow. If the value of `in` is too big, `balance_in + in` cannot fit in the bitvector (or an unsigned integer) without overflowing, and the assertion gets violated. You may check what happens if you try to put too big of a value in a bitvector below (note that the maximum value a bitvector of size 8 can store is 255).

In [ ]:
i = BitVecVal('259', 8)
i

3

Similar to Exercise 1, one of the vulnerable paths is, again, due to an integer overflow that can occur when we sum `balance_in` and `balance_out` at the end of the execution.
If we assume that this operation is not part of the analyzed smart contract key functionality, we can add an additional "no-overflow" constraint to the assertion. Same as in Exercise 1, our final condition will consist of two parts: *the sum of balances is less than 200 AND no overflow occured during the (balance_in + balance_out) operation*. In the absence of integer overflow, the following condition should hold: *the result of a sum is greater than or equal (`UGT`) to the first operand*.

**Task 12**: update the asserted condition to reflect the "no-overflow" constraint.

In [ ]:
"s_in = BitVec('in', 8)
s_out = BitVec('out', 8)
s_appr = BitVec('appr', 8)

bal_in = BitVecVal('100', 8)
bal_out = BitVecVal('100', 8)

s_in = BitVec('in', 8)
s_out = BitVec('out', 8)
s_appr = BitVec('appr', 8)

bal_in = BitVecVal('100', 8)
bal_out = BitVecVal('100', 8)

m = BV_SymCond([
    # (Line 1) uint balance_in = 100; uint balance_out = 100;
    PUSH(bal_in), PUSH(0), MSTORE(),    
    PUSH(bal_out), PUSH(1), MSTORE(),
    # (Line 3) if (_in >= _out && _out <= balance_out) {
    PUSH(s_out), PUSH(s_in), GTE(), PUSH(1), MLOAD(), PUSH(s_out), LTE(), AND(),
    # If the condition isn't satisfied, we jump to line 13
    PUSH(38),
    JUMPI(), # 15
    # (Line 4) if (_approved >= _in) {
    PUSH(s_in), PUSH(s_appr), GTE(),
    PUSH(32),
    JUMPI(), # 20
    # (Line 5) balance_in += _in;
    PUSH(0), MLOAD(), PUSH(s_in), ADD(), PUSH(0), MSTORE(),

    # (Line 6) _approved -= _in;
    PUSH(s_appr), PUSH(s_in), SUB(), PUSH(3), MSTORE(), # 31
  
    # (Line 11) balance_out -= _out;
    PUSH(s_out), PUSH(1), MLOAD(), SUB(), PUSH(1), MSTORE(), # MSTORE() is instruction #35

    # (Line 14) NOT (balance_in + balance_out >= 200);
    PUSH(0), MLOAD(), PUSH(1), MLOAD(), ADD(),
    PUSH(200), LTE(), NOT(),

    # TODO: (1) Add:
    # AND balance_in <= (balance_in + balance_out)
    ...
    ASSERT(),
    ])

m.run()

In [ ]:
#@title Task 12 Solution

s_in = BitVec('in', 8)
s_out = BitVec('out', 8)
s_appr = BitVec('appr', 8)

bal_in = BitVecVal('100', 8)
bal_out = BitVecVal('100', 8)


s_in = BitVec('in', 8)
s_out = BitVec('out', 8)
s_appr = BitVec('appr', 8)

bal_in = BitVecVal('100', 8)
bal_out = BitVecVal('100', 8)

m = BV_SymCond([
    # (Line 1) uint balance_in = 100; uint balance_out = 100;
    PUSH(bal_in), PUSH(0), MSTORE(),    
    PUSH(bal_out), PUSH(1), MSTORE(),
    # (Line 3) if (_in >= _out && _out <= balance_out) {
    PUSH(s_out), PUSH(s_in), GTE(), PUSH(1), MLOAD(), PUSH(s_out), LTE(), AND(),
    # If the condition isn't satisfied, we jump to line 13
    PUSH(38),
    JUMPI(), # 15
  # (Line 4) if (_approved >= _in) {
    PUSH(s_in), PUSH(s_appr), GTE(),
    PUSH(32),
    JUMPI(), # 20
    # (Line 5) balance_in += _in;
    PUSH(0), MLOAD(), PUSH(s_in), ADD(), PUSH(0), MSTORE(),

    # (Line 6) _approved -= _in;
    PUSH(s_appr), PUSH(s_in), SUB(), PUSH(3), MSTORE(), # 31
  
    # (Line 11) balance_out -= _out;
    PUSH(s_out), PUSH(1), MLOAD(), SUB(), PUSH(1), MSTORE(), # MSTORE() is instruction #35

    # (Line 14) ASSERT NOT (balance_in + balance_out >= 200);
    PUSH(0), MLOAD(), PUSH(1), MLOAD(), ADD(),
    PUSH(200), LTE(), NOT(),

    # TODO: (1) Add:
    # AND balance_in <= (balance_in + balance_out)
    PUSH(0), MLOAD(), PUSH(1), MLOAD(), ADD(),
    PUSH(0), MLOAD(), LTE(),
    AND(),
    ASSERT(),
    ])

m.run()

Constraints are satisfiable, assert can be violated
[in = 210, appr = 216, out = 22]
Constraints are not satisfiable, assert can't be violated
Constraints are satisfiable, assert can be violated
[in = 161, appr = 0, out = 70]


As you notice, the vulnerable path caused by an overflow is still present—in fact, it happens in the statement `balance_in += _in`.
As our final task in this exercise, let's update the code of `deposit` by inserting a overflow protection to this statement.

**Task 13**: update the opcode of the `deposit` function by adding an additional overflow check *before* `balance_in += _in` (you can add it after the statement execution too, but in this case you'll have to also reverse this memory update if the check fails).
The execution of the function should then jump right to the `assert` checking.

In [ ]:
s_in = BitVec('in', 8)
s_out = BitVec('out', 8)
s_appr = BitVec('appr', 8)

bal_in = BitVecVal('100', 8)
bal_out = BitVecVal('100', 8)

m = BV_SymCond([
    # (Line 1) uint balance_in = 100; uint balance_out = 100;
    PUSH(bal_in), PUSH(0), MSTORE(),    
    PUSH(bal_out), PUSH(1), MSTORE(),
    # (Line 3) if (_in >= _out && _out <= balance_out) {
    PUSH(s_out), PUSH(s_in), GTE(), PUSH(1), MLOAD(), PUSH(s_out), LTE(), AND(),
    # If the condition isn't satisfied, we jump to line 13
    PUSH(47),
    JUMPI(), # 15
    # (Line 4) if (_approved >= _in) {
    PUSH(s_in), PUSH(s_appr), GTE(),
    PUSH(41),
    JUMPI(), # 20
    # (Line 5) balance_in += _in;

    # TODO: (1) add the overflow check:
    # (balances_in + _in) <= balances_in
    # If condition is not satisfied: jump to line 13, i.e., assert
    # Else — continue with the memory update
    
    # (Line 6) _approved -= _in;
    PUSH(s_appr), PUSH(s_in), SUB(), PUSH(3), MSTORE(), # 40
  
    # (Line 11) balance_out -= _out;
    PUSH(s_out), PUSH(1), MLOAD(), SUB(), PUSH(1), MSTORE(), # MSTORE() is instruction #46

    # (Line 14) ASSERT NOT (balance_in + balance_out >= 200)
    # AND (balance_in <= balance_in + balance_out)
    PUSH(0), MLOAD(), PUSH(1), MLOAD(), ADD(),
    PUSH(200), LTE(), NOT(),
    PUSH(0), MLOAD(), PUSH(1), MLOAD(), ADD(),
    PUSH(0), MLOAD(), LTE(),
    AND(),
    ASSERT(),
    ])

m.run()

In [ ]:
#@title Task 13 Solution

s_in = BitVec('in', 8)
s_out = BitVec('out', 8)
s_appr = BitVec('appr', 8)

bal_in = BitVecVal('100', 8)
bal_out = BitVecVal('100', 8)

m = BV_SymCond([
    # (Line 1) uint balance_in = 100; uint balance_out = 100;
    PUSH(bal_in), PUSH(0), MSTORE(),    
    PUSH(bal_out), PUSH(1), MSTORE(),
    # (Line 3) if (_in >= _out && _out <= balance_out) {
    PUSH(s_out), PUSH(s_in), GTE(), PUSH(1), MLOAD(), PUSH(s_out), LTE(), AND(),
    # If the condition isn't satisfied, we jump to line 13
    PUSH(47),
    JUMPI(), # 15
    # (Line 4) if (_approved >= _in) {
    PUSH(s_in), PUSH(s_appr), GTE(),
    PUSH(41),
    JUMPI(), # 20
    # (Line 5) balance_in += _in;
    # And the overflow check:
    # (balances_in + _in) <= balances_in
    PUSH(0), MLOAD(), PUSH(s_in), ADD(), PUSH(0), MLOAD(), LTE(),
    # If condition is not satisfied: jump to line 13, i.e., assert
    PUSH(47),
    JUMPI(),
    # Else — continue with the memory update
    PUSH(0), MLOAD(), PUSH(s_in), ADD(), PUSH(0), MSTORE(),

    # (Line 6) _approved -= _in;
    PUSH(s_appr), PUSH(s_in), SUB(), PUSH(3), MSTORE(), # 40
  
    # (Line 11) balance_out -= _out;
    PUSH(s_out), PUSH(1), MLOAD(), SUB(), PUSH(1), MSTORE(), # MSTORE() is instruction #46

    # (Line 14) ASSERT NOT (balance_in + balance_out >= 200)
    # AND (balance_in <= balance_in + balance_out)
    PUSH(0), MLOAD(), PUSH(1), MLOAD(), ADD(),
    PUSH(200), LTE(), NOT(),
    PUSH(0), MLOAD(), PUSH(1), MLOAD(), ADD(),
    PUSH(0), MLOAD(), LTE(),
    AND(),
    ASSERT(),
    ])

m.run()

Constraints are not satisfiable, assert can't be violated
Constraints are not satisfiable, assert can't be violated
Constraints are satisfiable, assert can be violated
[in = 161, appr = 0, out = 70]
Constraints are not satisfiable, assert can't be violated


Now the only vulnerable path being reported indeed corresponds to the case when the user "sends in" a sufficient amount of `in` tokens but does not approve enough of them to be transferred to the contract.


### Additional Exercises 
Note that in this exercise we intentionally limited the number of EVM opcode used to make it less heavy on the EVM internals. However, the list of opcodes and an interactive playground can be found on the [evm.codes](https://www.evm.codes/) website. We leave optimizing the opcode representation we've used throughout the exercise to you.

As another exercise, you can also develop a similar symbolic execution tool with operational semantics for a high-level smart contract language such as Solidity or Vyper.

## References
- Philip Zucker's "Z3 Tutorial" https://colab.research.google.com/github/philzook58/z3_tutorial/blob/master/Z3%20Tutorial.ipynb
- William Berman's "EVM Symbolic Execution" https://github.com/williamberman/evm-symbolic-execution/blob/master/EVM%20Symbolic%20Execution.ipynb
- rise4fun https://rise4fun.com/z3/tutorialcontent/guide
- https://ericpony.github.io/z3py-tutorial/guide-examples.htm
- Programming Z3 - https://theory.stanford.edu/~nikolaj/programmingz3.html
- Nikolaj Bjorner's tutorial https://youtu.be/nGwyNmsxX6I
- Hakank's examples http://www.hakank.org/z3/
- Yurichev's book "SMT by Example" https://yurichev.com/writings/SAT_SMT_by_example.pdf
- http://hackage.haskell.org/package/sbv
- https://www.youtube.com/watch?v=ruNFcH-KibY Tikhon Jelvis - Analyzing Programs with Z3
- https://www.youtube.com/watch?v=rvPWDgJc0O4&ab_channel=ACMSIGPLAN - Nadia Polykarpova on Z3
- SAT SMT school https://sat-smt.in/
- Emina Torlak's course https://courses.cs.washington.edu/courses/cse507/19au/calendar.html
- Lindsey Kuper - SMT Solving and Solver-Aided Systems http://composition.al/CSE290Q-2019-09/
- http://www.sc-square.org/CSA/school/lectures.html